Ordering and Delivery to Google Earth Engine (GEE)

Based off of the client's example (https://github.com/planetlabs/planet-client-python/blob/main/examples/orders_create_and_download_multiple_orders.py)

In [1]:
import planet

Define order

In [2]:
iowa_aoi = {
    "type":
    "Polygon",
    "coordinates": [[[-91.198465, 42.893071], [-91.121931, 42.893071],
                     [-91.121931, 42.946205], [-91.198465, 42.946205],
                     [-91.198465, 42.893071]]]
}

iowa_images = ['20200925_161029_69_2223', '20200925_161027_48_2223']

Define cloud delivery location

In [3]:
delivery_config = planet.order_request.delivery(
    cloud_config=planet.order_request.google_earth_engine(
        project='planet-devrel-dev', collection='gee-integration-testing'))
print(delivery_config)

{'google_earth_engine': {'project': 'planet-devrel-dev', 'collection': 'gee-integration-testing'}}


Build order request

In [4]:
data_products = [
    planet.order_request.product(iowa_images, 'analytic_udm2', 'PSScene')
]
tools = [planet.order_request.clip_tool(iowa_aoi)]

iowa_order = planet.order_request.build_request(name='iowa_order',
                                                products=data_products,
                                                delivery=delivery_config,
                                                tools=tools)

print(iowa_order)

{'name': 'iowa_order', 'products': [{'item_ids': ['20200925_161029_69_2223', '20200925_161027_48_2223'], 'item_type': 'PSScene', 'product_bundle': 'analytic_udm2'}], 'delivery': {'google_earth_engine': {'project': 'planet-devrel-dev', 'collection': 'gee-integration-testing'}}, 'tools': [{'clip': {'aoi': {'type': 'Polygon', 'coordinates': [[[-91.198465, 42.893071], [-91.121931, 42.893071], [-91.121931, 42.946205], [-91.198465, 42.946205], [-91.198465, 42.893071]]]}}}]}


Define create and download order functions

In [5]:
async def create_and_get_order(order_detail, client):
    with planet.reporting.StateBar(state='creating') as reporter:
        # create
        order = await client.create_order(order_detail)
        reporter.update(state='created', order_id=order['id'])
        # wait for completion
        await client.wait(order['id'], callback=reporter.update_state)

    order_details = await client.get_order(order_id=order['id'])
    return order_details

Create and download order

In [6]:
async with planet.Session() as ps:
    client = planet.OrdersClient(ps)
    order_details = await create_and_get_order(iowa_order, client)

05:06 - order cd214ce6-d3e3-46eb-a852-74d4f4899443 - state: success


In [7]:
print(order_details)

{'_links': {'_self': 'https://api.planet.com/compute/ops/orders/v2/cd214ce6-d3e3-46eb-a852-74d4f4899443', 'results': [{'delivery': 'success', 'expires_at': '2022-11-15T18:22:49.403Z', 'location': 'https://api.planet.com/compute/ops/download/?token=eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE2Njg1MzY1NjksInN1YiI6IjZoNGl1RDVFY2dMbnh0WkE1ZzErZTNrMnJxOCttMFRuemx1S1QzSzBIQTNleUtVT3pXaFd0aTNuZFNZSTZxd0dWbUN4SHozdmVGSzEvYTVydGJmdkFBPT0iLCJ0b2tlbl90eXBlIjoiZG93bmxvYWQtYXNzZXQtc3RhY2siLCJhb2kiOiIiLCJhc3NldHMiOlt7Iml0ZW1fdHlwZSI6IiIsImFzc2V0X3R5cGUiOiIiLCJpdGVtX2lkIjoiIn1dLCJ1cmwiOiJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY29tcHV0ZS1vcmRlcnMtbGl2ZS9jZDIxNGNlNi1kM2UzLTQ2ZWItYTg1Mi03NGQ0ZjQ4OTk0NDMvUFNTY2VuZS8yMDIwMDkyNV8xNjEwMjdfNDhfMjIyM19tZXRhZGF0YS5qc29uP1gtR29vZy1BbGdvcml0aG09R09PRzQtUlNBLVNIQTI1Nlx1MDAyNlgtR29vZy1DcmVkZW50aWFsPWNvbXB1dGUtZ2NzLXN2Y2FjYyU0MHBsYW5ldC1jb21wdXRlLXByb2QuaWFtLmdzZXJ2aWNlYWNjb3VudC5jb20lMkYyMDIyMTExNCUyRmF1dG8lMkZzdG9yYWdlJTJGZ29vZzRfcmVxdWVzdFx1MDAyNlgtR29vZy1